In [2]:
import os
import re
import nltk
import numpy as np
from sklearn import feature_extraction
from tqdm import tqdm
import pandas as pd
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [84]:
dataset=pd.read_csv('train_data.csv')
df=dataset[:]

In [85]:
def textlist2wordslist(textlist):
    docs1 = []
    for doc_raw in textlist:
        doc = nltk.word_tokenize(str(doc_raw))
        docs1.append(doc) 
    docs2 = [[w.lower() for w in doc] for doc in docs1]
    docs3 = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs2]
    
    
    stop_list = stopwords.words('english')
     
    
    docs4 = [[w for w in doc if w not in stop_list] for doc in docs3]
    
    stemmer = PorterStemmer()
    docs5 = [[stemmer.stem(w) for w in doc] for doc in docs4]
    return docs5

In [86]:
text1=df['Headline'].tolist()
text2=df['articleBody'].tolist()
clean_text=textlist2wordslist(text1+text2)

In [87]:
dictionary = corpora.Dictionary(clean_text)
corpus = [dictionary.doc2bow(doc) for doc in clean_text]

In [88]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=9)
lda_model.save("lda_model")
#lda_disk=gensim.models.ldamodel.LdaModel.load("lda_model1")

D:\anaconda\installer\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [89]:
df.head()

,Headline,Body ID,Stance,articleBody,Headline_processed,Body_processed
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\r\n...,"['polic', 'find', 'mass', 'grave', 'least', 'b...","['danni', 'boyl', 'direct', 'untitl', 'film', ..."
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,"['hundr', 'palestinian', 'flee', 'flood', 'gaz...","['hundr', 'palestinian', 'evacu', 'home', 'sun..."
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,"['christian', 'bale', 'pass', 'role', 'steve',...","['moscow', 'resid', 'hospit', 'wound', 'intim'..."
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,"['hbo', 'appl', 'talk', 'appl', 'tv', 'stream'...","['reuter', 'canadian', 'soldier', 'shot', 'can..."
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...","['spider', 'burrow', 'tourist', 'stomach', 'ch...","['fear', 'arachnophob', 'stori', 'bunburi', 's..."


In [90]:
import numpy as np
import pandas as pd

global lda_model,dictionary,corpus

def dominant_topic(corpus):
    lda=lda_model[corpus]
    topic=[]
    for i, row in enumerate(lda):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        topic_num, Percent=row.pop(0)
        topic.append(topic_num)
    return topic

In [91]:
dom_topic=dominant_topic(corpus)
size=len(corpus)//2
headline_topic=dom_topic[:size]
body_topic=dom_topic[size:]

In [92]:
df['headline_topic']=pd.DataFrame(data={'headline_topic':headline_topic})
df['body_topic']=pd.DataFrame(data={'body_topic':body_topic})
df.head()

D:\anaconda\installer\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\anaconda\installer\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Headline,Body ID,Stance,articleBody,Headline_processed,Body_processed,headline_topic,body_topic
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\r\n...,"['polic', 'find', 'mass', 'grave', 'least', 'b...","['danni', 'boyl', 'direct', 'untitl', 'film', ...",0,3
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,"['hundr', 'palestinian', 'flee', 'flood', 'gaz...","['hundr', 'palestinian', 'evacu', 'home', 'sun...",3,3
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,"['christian', 'bale', 'pass', 'role', 'steve',...","['moscow', 'resid', 'hospit', 'wound', 'intim'...",3,1
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,"['hbo', 'appl', 'talk', 'appl', 'tv', 'stream'...","['reuter', 'canadian', 'soldier', 'shot', 'can...",0,1
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...","['spider', 'burrow', 'tourist', 'stomach', 'ch...","['fear', 'arachnophob', 'stori', 'bunburi', 's...",1,1


In [93]:
df['topic_matching']=df.apply(lambda x: 1 if x['headline_topic']==x['body_topic'] else 0, axis=1)
df.head()

D:\anaconda\installer\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Headline,Body ID,Stance,articleBody,Headline_processed,Body_processed,headline_topic,body_topic,topic_matching
0,Police find mass graves with at least '15 bodi...,712,unrelated,Danny Boyle is directing the untitled film\r\n...,"['polic', 'find', 'mass', 'grave', 'least', 'b...","['danni', 'boyl', 'direct', 'untitl', 'film', ...",0,3,0
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree,Hundreds of Palestinians were evacuated from t...,"['hundr', 'palestinian', 'flee', 'flood', 'gaz...","['hundr', 'palestinian', 'evacu', 'home', 'sun...",3,3,1
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated,30-year-old Moscow resident was hospitalized w...,"['christian', 'bale', 'pass', 'role', 'steve',...","['moscow', 'resid', 'hospit', 'wound', 'intim'...",3,1,0
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated,(Reuters) - A Canadian soldier was shot at the...,"['hbo', 'appl', 'talk', 'appl', 'tv', 'stream'...","['reuter', 'canadian', 'soldier', 'shot', 'can...",0,1,0
4,Spider burrowed through tourist's stomach and ...,1923,disagree,"Fear not arachnophobes, the story of Bunbury's...","['spider', 'burrow', 'tourist', 'stomach', 'ch...","['fear', 'arachnophob', 'stori', 'bunburi', 's...",1,1,1


In [94]:
col=['Headline_processed','Body_processed','headline_topic','body_topic']

In [95]:
df=df.drop(columns=col)
df.to_csv('topic_feature.csv')